# Inteligencia artificial

La inteligencia artificial (IA) hace posible que las máquina aprendan de la experiencia, se ajusten a nuevas aportaciones y realicen tareas como hacen los humanos. La mayoría de los ejemplos de inteligencia artificial de los que usted escucha hoy día – desde computadoras que juegan ajedrez hasta automóviles que se conducen por sí solos – se sustentan mayormente en aprendizaje a fondo (deep learning) y procesamiento del lenguaje natural. Mediante el uso de estas tecnologías, las computadoras pueden ser entrenadas para realizar tareas específicas procesando grandes cantidades de datos y reconociendo patrones en los datos.

![c3_4.png](https://drive.google.com/uc?export=view&id=1e8cKqzwywt1OKqW1rfLLT6chdSuB1Hx_)

# Web Scraper

Cuando los datos no los tienes en una base de datos sino están colgados en una web y ni siquiera hay API no queda otra que extraerlos directamente de la web. Eso es el web scraping, básicamente vamos a una URL y extraemos de su código HTML lo que queramos. Esto da solución, las API y el web scraping, al problema de los datos, que en su mayoría son **no estructurados**, es decir, no nos vienen dados con un formato csv, txt o excel de una web, sino vienen en bruto y tenemos que hacer todo el proceso de manejarlos nosotros. Esto viene dado porque mucho de los datos actuales proceden de redes sociales (ya sean las más convencionales, Instagram, Twitter, Tik tok, o de otras como Tripadvisor, Booking, Airbnb, etc.)

![c3_4.png](https://drive.google.com/uc?export=view&id=1L7d0ehVPmgjzs--od-5r8bDw6HGZHwsb)

Se puede decir que también es un Bot, porque simula una acción humana. Es decir, estamos programando a una máquina para hacer algo que podríamos hacer manualmente pero muy rápido.

![hero-recaptcha-demo.gif](https://drive.google.com/uc?export=view&id=16zrPHCLOVAoQA74B_U-tcCo4kJK3s5H1)

Vamos a usar técnicas de web scraping para poder obtener los comentarios de los clientes de Tripadvisor, con el fin posterior de clasificar si su comentario es bueno o negativo a posteriori.

![c3_4.png](https://drive.google.com/uc?export=view&id=1mdVm72iV5rG1ZseWltnQ-ql6NwJyBP3E)


Como Tripadvisor no nos deja ninguna API para poder descargarlo tenemos que optar a este tipo de técnicas. En concreto hemos cogido dos restaurantes al azar de la isla y los hemos incluido para su scrapeo.

Importante en este caso que nos enfrentamos a un código más complejo, formado por una clase que se encargará de las funcionalidades y por otro lado vemos el
```if __name__ == '__main__':``` que utilizado cuando quieres ejecutar un programa Python. Para los que saben otros lenguajes de programación es la función main. Cuando se ejecute desde terminal ```programa.py``` va a buscar a esta función. En concreto para ejecutar un bloque en Jupyter no haría falta, pero viene bien que vean el ejemplo de como se haría de verdad en un script *.py* 

Para el scraping se va a usar **selenium** que descarga todo el HTML de la página que le indiquemos, y **BeautifulSoup** que coge ese HTML y lo transforma a un formato donde será capaz de buscar las etiquetas que le indiquemos con el nombre de algún campo si hiciera falta.

HTML es un lenguaje de etiquetas, donde se ponen elementos según su utilidad. Ejemplo \<a>\</a> sirve para poner un enlace \<table>\</table> una tabla o \<p>\</p> un párrafo.

![etiqueta_html.png](https://drive.google.com/uc?export=view&id=1wpW1pDIyXTbLIl6MzX3haPTnBmD-fvTx)

Para ver el código html de cualquier página web, apretar boton derecho la web y apretar inspeccionar. Si no nos sale por defecto, apretar la pestaña de "elementos"

![inspeccionar.png](https://drive.google.com/uc?export=view&id=1dQX42MAV8RXmIxbOdN33kBP1QooTDn9G)


El programa va a acceder a las páginas del restaurante que le indiquemos. Para eso le indicamos código de zona y código de restaurante. Con eso formamos la URL, porque en concreto en tripadvisor se repite que las webs van a estar formadas por"https://www.tripadvisor.co.uk/Restaurant_Review-{código_zona}-{código_restaurante}-Reviews-or{número_página_comentario}".

Cada página tiene diez comentarios visualizados. Por ejemplo, en la que tenega "número_página_comentario" igual a 0 se mostrarán los comentarios de 0 a 9. Cuando sea 10, de 10 a 20. Y así sucesivamente.

Desde la página principal del restaurante no se visualiza todo el comentario completo, por eso accedemos a otra página luego, la propia del comentario. Ahí sí podremos buscar el campo donde está el texto implícito del comentario y almacenarlos en nuestro objeto pandas que habremos creado previamente.

Las clases tienen variables propias llamadas self. Son variables que podrémos llamar en todo el ámbito de la clase y que se inicializan en el *__init__()*. Cuando creamos una clase estamos llamando a *__init__* implícitamente. Por eso le pasamos argumentos, porque en *__init__()* los estamols pidiendo. Luego, tenemos que acceder a sus métodos manualmente, por ejemplo en este caso tenemos **scraperTrip.get_comentarios()**. Siempre que hemos visto en librerías colocar, por ejemplo con pandas, *pd.read_csv()*, entrábamos a un método de una clase.

Al final obtendremos un Dataframe de Pandas con los datos de los comentarios de los hoteles que hayamos indicado. Sus columnas serán:
 - cod_zona. Código de zona.
 - cod_rest. Código de restaurante.
 - cods_revs. Código de comentario.
 - revs. Comentario.

La forma en la que comento la funcionalidad de cada función es el estandar no oficial de Python (acuérdense que en Python no hay normas escritas sino recomendaciones). 


In [1]:
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [5]:
# import urllib.request
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time


# Todos los restaurantes y comentarios de los propios tienen el mismo formato de URL

class scraperTripadvisorRestaurantes():

    def __init__(self, lista_restaurantes):
        """
            :params lista_restaurantes:   lista con todos los códigos de 
                                          restaurantes y sus zonas que vamos
                                           a visitar.

            Inicializo la clase. Aprovechamos que en este caso se repite siempre
            la estructura de los restaurantes para poner la URL con llaves que 
            luego sustituiremos con el método "format()"

            - El primer corchete corresponde al código de la zona
            - El segundo corchete al código del restaurante
            - El tercer corchete, en el caso de los restaurantes, 
              se refiere a la página de los comentarios.
            - El tercer corchete, en el caso de la URL de las reviews,
              al del comentario.
        """
        # Definimos las bases que vamos a usar para URL de restaurantes y de
        # revisiones
        self.URL_base_rest = "https://www.tripadvisor.co.uk/Restaurant_Review-{}-{}-Reviews-or{}"
        self.URL_base_rev = "https://www.tripadvisor.co.uk/ShowUserReviews-{}-{}-{}"
        self.lista_restaurantes = lista_restaurantes
        # Inicializamos lista con los restaurantes
        self.URL_rest = []
        # Añadir los restaurantes
        for restaurante in lista_restaurantes:
            # Al inicio todas las páginas empiezan por la 0
            self.URL_rest.append(self.URL_base_rest.format(restaurante['cod_zona'], 
                                                           restaurante['cod_rest'],
                                                           0))
        # Inicializamos el DataFrame que vamos a devolver.
        # En este caso empezará vacío, solo indicaremos las columnas
        self.revs = pd.DataFrame(columns=['cod_zona','cod_rest',
                                          'cods_revs','revs'])

    def get_html_web(self, url):
      """
        Devolver el código html.
        Se obtiene código con la librería selenium y se transforma a un objeto
        BeatifulSoup para poder tratarlo luego.
        :arg url: Url que se desea obtener
        :return soup: código html parseado por la librería BeatifulSoup.
      """      
      chrome_options = webdriver.ChromeOptions()
      chrome_options.add_argument('--headless')
      chrome_options.add_argument('--no-sandbox')
      chrome_options.add_argument('--disable-dev-shm-usage')
      driver = webdriver.Chrome('chromedriver',options=chrome_options)
      driver.get(url)

      soup = ''
      while soup == '':
        time.sleep(0.1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

      driver.quit()

      return soup

    def get_comentarios(self):
        """
          Devolver comentarios
          :return comentarios: diccionario con todos los comentarios 
                               por restaurantes
        """
        # Obtenemos lista de ids de comentarios que guardamos en self.URL_rev
        self.get_ids_comentarios()

        print('Añadir comentario en sí desde su dirección concreta')
        # ===================================================
        # ===================    3    =======================
        # ====== Obtenemos los comenarios en sí mismos ======
        # ===================================================
        for i in range(0,len(self.revs)):


            # Obtenemos todo el html de la página
            url = self.URL_base_rev.format(self.revs['cod_zona'].iloc[i],
                                           self.revs['cod_rest'].iloc[i],
                                           self.revs['cods_revs'].iloc[i])
            print('pagina del comentario',url)
            # Inicializamos el soup para seleccionar etiquetas
            soup = self.get_html_web(url)
            comentario_campo = soup.findAll("p", {"class": "partial_entry"})
            comentario_texto = comentario_campo[0].text

            self.revs['revs'].iloc[i] = comentario_texto
            
        return self.revs

    def get_ids_comentarios(self):
        """
            Devuelve los id de los comentarios que aparecen en cada página de
            los restaurantes y con los que se puede saber la página URL
            para acceder


        """
        for i in range(0, len(self.URL_rest)):
            print('Restaurante url:', self.URL_rest[i])
            # Obtenemos todo el html de la página e inicializamoz 
            # el soup para seleccionar etiquetas
            soup = self.get_html_web(self.URL_rest[i])
            

            # ===================================================
            # ===================    1    =======================
            # = Obtenemos el número de páginas con comentarios ==
            # ===================================================

            # Si no hay páginas entonces lo indicamos
            try:
                n_comentarios_elemento = soup.findAll("div", {"class": "ui_header h4 counts"})

                # Coger el texto de ese elemento el primero, y del "X results" 
                # que obtenga solo el "X"
                n_comentarios= n_comentarios_elemento[0].text.split()[0]

                # Redondeamos a la decena más cercana, pues cada bloque de
                # comentarios viene de diez en diez. Por ello también
                # dividimos, pues queremos saber el número de páginas
                # de cometarios
                n_pag_comentarios = int(round(int(n_comentarios),-1)/10)

            except:
                n_pag_comentarios = 0

            # ===================================================
            # ===================    2    =======================
            # ===  Obtenemos las páginas de cada comentario =====
            # ===================================================
            for n_pag_comentario in range(0, n_pag_comentarios):
                  
                # Obtenemos todos los campos con comentarios
                comentarios = soup.findAll("div", {"class": "review-container"})
                print('Página',n_pag_comentario+1,'de', n_pag_comentarios,'comentarios')

                # Obtenemos todos los campos con comentarios
                for comentario in comentarios:
                    # Obtenemos todo el html de la página
                    url = self.URL_base_rest.format(self.lista_restaurantes[i]['cod_zona'],
                                                    self.lista_restaurantes[i]['cod_rest'],
                                                    n_pag_comentario)

                    # Inicializamoz el soup para seleccionar etiquetas
                    soup = self.get_html_web(url)
                    
                    # Vamos a obtener sólo el id del comentario
                    # porque aquí no lo podemos descargar enteros desde aquí.
                    # Este es el campo donde está el enlace y el id
                    campo_enlace = comentario.findChildren("a" ,{"class":"title"})
                    # Obtengo y guardo id comentarios para luego buscarlo
                    id = campo_enlace[0].get('id')
                    # El id comentario tiene forma por ejemplo "rn778259474", pero
                    # nosotros solo queremos "r778259474".
                    # Todos empiezan "rn" más el número, así que eliminamos la n
                    id = id.replace('n', '')
                    self.revs=self.revs.append({'cod_zona':self.lista_restaurantes[i]['cod_zona'], 
                                                'cod_rest':self.lista_restaurantes[i]['cod_rest'], 
                                                'cods_revs':id, 
                                                'revs':''} , ignore_index=True)

                print('Comentarios',len(self.revs))
                

# # Ejecutar la aplicación principal
if __name__ == '__main__':  # Función principal del programa
     # Vamos a sacar dos restaurantes en concreto.
     # Se podría también hacer un scraper de todos los restaurantes 
     lista_restaurantes = [
                           {'cod_zona':'g662606', 'cod_rest':'d14091847'},
                           {'cod_zona':'g796999', 'cod_rest':'d15662340'},
                           {'cod_zona':'g659661', 'cod_rest':'d15677484'},
                           {'cod_zona':'g659661', 'cod_rest':'d13169911'}
                          ]
     scraperTrip = scraperTripadvisorRestaurantes(lista_restaurantes)
     df_comentarios = scraperTrip.get_comentarios()
     print(df_comentarios)


Restaurante url: https://www.tripadvisor.co.uk/Restaurant_Review-g662606-d14091847-Reviews-or0
Página 1 de 6 comentarios
Comentarios 10
Página 2 de 6 comentarios
Comentarios 20
Página 3 de 6 comentarios
Comentarios 30
Página 4 de 6 comentarios


KeyboardInterrupt: 

In [6]:
# Si imprimimos el DataFrame
df_comentarios

NameError: name 'df_comentarios' is not defined

Podemos si queremos almacenar un csv de la siguiente manera. 

Si no estuvieramos en Colab bastaría con poner df_comentarios.to_csv('nombre_archivo.csv'). En el resto de líneas estamos pidiendo permisos para poder almacenarlo en nuestro Drive.

In [ ]:
# Guardar CSV en el Colab
# Tienes que pinchar el enlace que te sale, aceptar con tu cuenta y después copiar
# la salida que te da y pegarla en el recuadro.
from google.colab import drive
drive.mount('drive')
df_comentarios.to_csv('comentarios_tripadvisor130.csv')
!cp comentarios_tripadvisor130.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


Por si va muy lento la extracción dejo el csv resultante subido. Lo dejo comentado por si les da tentación de ejecutarlo sin leer y no les vaya a fastidiar.


![c3_4.png](https://drive.google.com/uc?export=view&id=1Ho7ck3Wbt1TpwFonRtLqZRoJRijB8dwI)

PD: Acuérdense de que para leer excel el comando con Pandas es
pd.read_excel() en vez de read_csv(), que en las clases solo hemos visto el caso de csv.

In [ ]:
# Si quieren leer directamente el csv, 
df_comentarios = pd.read_csv('https://raw.githubusercontent.com/SerDiaz/Introduccion_programacion_en_python/main/dia5/comentarios_tripadvisor130.csv', index_col=0)

### Ejercicio 1
Prueba a buscar dos restaurantes más y añadirlos al scrapeo y guardar el nuevo csv generado en tu ordenador (con que me pongan el código que tendrían que usar me vale, que ya sabemos que el ejecutar tarda. Aunque si lo quieren probar mientras explico mejor).

In [ ]:
# Resolver

### Ejercicio 2
En este ejercicio vamos describir como realizar un scrapeo. Como si fuéramos detectives.

![conan.png](https://drive.google.com/uc?export=view&id=1P2sKXmNI1cqh-ycq1jM7bzOkpVu5gHh6)

Para ejemplificar lo que me refiero, voy a describir como quedaría explicado el proceso que hemos llevado a cabo para obtener los comentarios de los restaurantes de tripadvisor con lenguaje natural: 

**1 )** Accedo a la web de un restaurante cualquiera, por ejemplo https://www.tripadvisor.co.uk/Restaurant_Review-g662606-d14091847-Reviews-Restaurante_Donaire-Costa_Adeje_Adeje_Tenerife_Canary_Islands.html

**2)** Los comentarios se encuentran al final de la página. En concreto vienen divido por páginas por las que puedo navegar interaccionando con los botones dispuestos. En cada página hay un máximo de 10 comentarios.

**3)** Al apretar cada página de comentarios cambia la URL. Por tanto no tengo que simular los clicks con Selenium, sólo acceder a la URL. 

   Ejemplo, la URL para los comentarios de 10 al 20 es https://www.tripadvisor.es/Restaurant_Review-g662606-d14091847-Reviews-or10-Restaurante_Donaire-Costa_Adeje_Adeje_Tenerife_Canary_Islands.html, para los siguientes 10 https://www.tripadvisor.es/Restaurant_Review-g662606-d14091847-Reviews-or20-Restaurante_Donaire-Costa_Adeje_Adeje_Tenerife_Canary_Islands.html

**4)** Comparando la URL con la de otros restaurantes, puedo deducir que lo único que cambia de las URL, además del nombre al final, es el "g662606" y el "d14091847", además del "or20" que nos indica la página de comentarios actual. 

   Testeo que solo escribiendo hasta el "or20" ya me reedirige al restaurante, evitando tener que escribir el "Restaurante_Donaire-Costa_Adeje_Adeje_Tenerife_Canary_Islands.html
", por lo que el código puedo hacerlo más elegante. 

   También comparando con otros restaurante acabo deduciendo que el "g662606" es el código de la zona, y que el "d14091847" es el código del restaurante.

**5)** Clickeando cada comentario te lleva a una página con una url única como esta: https://www.tripadvisor.co.uk/ShowUserReviews-g662606-d14091847-r812486250-Restaurante_Donaire-Costa_Adeje_Adeje_Tenerife_Canary_Islands.html

Esta url comparte el código de zona y de restaurante anterior, pero añade un nuevo campo, el "r812486250", que después de comparar con otros comentarios, entiendo que es un identificador único.

Por tanto, para guardar todos los comentarios, sólo necesito guardar este identificador.

**6)** La cadena por tanto será la siguiente: 

    a) Abrir la página del restaurante. 

    b) Obtener la página de cada comentario.

    c) Cambiar de página de comentarios abriendo la siguiente página.
    
    d) Repetir el proceso hasta tener almacenado todas las páginas de comentarios para cada restaurante buscado.

    e) Abrir cada página de comentarios y extraer el comentario en sí, con los identificadores correspondiente 

**7)** Ahora especifico las etiquetas en sí mismo a extraer. Buscando en el código html de la página el identificador "812486250", me doy cuenta que hay varios bloques donde puedo conseguir el identificador. Me voy a decantar por buscar todas las etiquetas **a** donde el nombre del atributo **class** sea **title**. En la misma etiqueta tenemos otro atributo **id** donde viene el identificador, aunque en este caso escrito como "rn812486250" en lugar de "r812486250". Así que tenemos que borrar la "n". Con eso tendremos los comentarios.

**8)** Para cada página, puedo hacerlo de varias formas. En mi caso, para hacer un bucle, necesito saber la última página de comentarios. Sólo se puede interactuar con 6 páginas. Si hay más páginas es imposible saber el fin a no ser que interactúe.

En un lado de la página aparecen el número de reviews, pero veo que no cuadra con el número de comentarios visibles (entiendo que están mostrando sólo los últimos comentarios desde cierta fecha). En código HTML sí encuentro un lugar donde dice exactamente el número de comentarios visibles "58 results". Sabía que eran 58 porque en la página 6, la última, sólo había 8 comentarios visibles. el "58 results" está en la etiqueta **div** cuyo atributo **class** es **ui_header h4 counts**. Una vez obtenido el número "58", solo tengo que redondear la decena a "60", y dividir por 10 para saber el número de páginas totales y hacer el bucle.

**9)** Para obtener los comentarios en sí hago el inspeccionar, en la página de cada comentario, el texto del comentario. Me fijo que el comentario está en la etiqueta **p** cuyo atributo **class** tiene por nombre **partial_entry**. 



### **Ahora, el ejercicio a realizar ustedes consiste en anotar los pasos necesarios para lograr realizar el scraping de todos los restaurantes de la isla de Tenerife**. 

 --------------- Doble click aquí y resolver -------------------------



# Machine learning y deep learning

![c3_4.png](https://drive.google.com/uc?export=view&id=1zSeqmsj2ROwugHEmQQthSxVVhUGGphrB)

El machine learning –aprendizaje automático– es una rama de la inteligencia artificial que permite que las máquinas aprendan sin ser expresamente programadas para ello. Una habilidad indispensable para hacer sistemas capaces de identificar patrones entre los datos para hacer predicciones. Esta tecnología está presente en un sinfín de aplicaciones como las recomendaciones de Netflix o Spotify, las respuestas inteligentes de Gmail o el habla de Siri y Alexa.

El machine mearning es un maestro del reconocimiento de patrones, y es capaz de convertir una muestra de datos en un programa informático capaz de extraer inferencias de nuevos conjuntos de datos para los que no ha sido entrenado previamente.

## Deep learning

![c3_4.png](https://drive.google.com/uc?export=view&id=1TYQM0ph-4If3VLqsOBBjSWOwrhChNcYE)

Deep Learning es uno de los métodos de aprendizaje de la inteligencia artificial, y a día de hoy pertenece a un subcampo  del Machine Learning.

El Deep Learning o aprendizaje profundo se define como un algoritmo automático estructurado o jerárquico que emula el aprendizaje humano con el fin de obtener ciertos conocimientos. Destaca porque no requiere de reglas programadas previamente, sino que el propio sistema es capaz de «aprender» por sí mismo para efectuar una tarea a través de una fase previa de entrenamiento.

A su vez, también se caracteriza por estar compuesto por redes neuronales artificiales entrelazadas para el procesamiento de información. Se emplea principalmente para la automatización de análisis predictivos.

 

Los algoritmos que componen un sistema de aprendizaje profundo se encuentra en diferentes capas neuronales compuestas por pesos (números). El sistema está dividido principalmente en 3 capas:

 

- Capa de entrada (Intup Layer): Está compuesto por las neuronas que asimilan los datos de entrada, como por ejemplo imagen o una tabla de datos.

 

 

- Capa oculta (Hidden Layer): Es la red que realiza el procesamiento de información y hacen los cálculos intermedios. Cada más neuronas en esta capa haya, más complejos son los cálculos que se efectúan.

 

 

- Salida (Output Layer): Es el último eslabón de la cadena, y es la red que toma la decisión o realiza alguna conclusión aportando datos de salida.

 

Antes de empezar con el código y las muestras hay que tener claro varias cosas. Primero que hay dos formas de predecir/clasificar datos. Por un lado tenemos aprendizaje supervisado, cuando tenemos muestras previamente clasificadas y se lo proporcionamos al algoritmo para que aprenda de ellas y lo asocie a los datos que queramos, y por otro tenemos el aprendizaje no supervisado, cuando no le damos ninguna clasificación previa sino dejamos al algoritmo "jugar" y buscar patrones por sí mismo.

![c3_4.png](https://drive.google.com/uc?export=view&id=1GAyoMSgYDPfnSK1lBJdtk30sx_ZOzjhi)

Ahora abajo un ejemplo de una red neuronal profunda. Que acuérdense, no deja de ser un tipo de algoritmo de Machine Learning. Ásí que nuestro modelo de machine learning podría ser desde una regresión lineal, hasta este tipo de modelos. Al final no son tan mágicos como se cree, solo son fórumlas matemáticas, no tan complejas como se puede pensar, repetidas muchas veces de forma muy rápida. Ahí reside la verdadera magia, en la capacidad de los ordenadores actuales de ejecutarlo, pues muchos de los modelos usados actualmente recientemente son bastante más viejos que nosotros.

![c3_4.png](https://drive.google.com/uc?export=view&id=1bmhg4uUrUhPxf37plfWSepjUuuhMXE-l)

Para este proceso necesitaremos primero una fichero con datos clasificados previamente. En este caso vamos a usar uno que me encontré por ahí, pero como veremos después no nos va muy allá. 

Podríamos clasificar nosotros nuestras propias muestras también manualmente en un excel, diciendo si tales comentarios son positivos o negativos. Pero si quieres tener un buen modelo hablamos de miles, decenas, cientos e incluso millones de datos. Como entenderan, un trabajo laborioso donde los haya.

Como siempre, para cargar un csv e imprimimos los datos para verlos.

Aquí es fácil, tenemos las review por un lado y por otro la columna Liked donde 1 significa que valoración positiva y 0 que no.

El entrenamiento consiste en una serie de etapas que se repiten una y otra vez permitiendo al algoritmo buscar patrones en los datos que le pasemos y mejorando el accuracy que nos indica cuánto de bueno es nuestra modelo (es decir, el número de predicciones acertadas que ha logrado obtener). En medio del entreno también le podemos pasar un conjunto de validación, si deseamos. Esto es un conjunto de datos que no son utilizados en el cálculo directo de los pesos.

Tanto se use o no un conjunto de datos de validación, al finalizar el entrenamiento hay que cotejarlo con otros datos que no se hayan usado para entrenar y comprobar que el algoritmo funciona correctamente. Si no corremos el riesgo de que el algoritmo aprenda los patrones exactos de los datos pasados pero no sepa generalizar. Eso se llama sobre entrenamiento.

![c3_4.png](https://drive.google.com/uc?export=view&id=1t15BsTaM05dALwBgOr1WfnYxVLFqUqO6)

In [8]:
import tensorflow as tf
import os
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import warnings
import matplotlib.pyplot as plt

In [9]:
# Importing the libraries
import numpy as np
import pandas as pd
dataset = pd.read_csv('https://raw.githubusercontent.com/SerDiaz/Introduccion_programacion_en_python/main/dia5/Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)
dataset

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [16]:
# Esta función nos servirá para transformar los comentarios, que están en texto, en números.
# Necesario para poder entrenar a los modelos.
# También cualquier comentario que queramos clasificar una vez hecho el modelo
# lo tendremos que clasificar
def get_data_x_y(dataset, review_field, train=False):
    corpus = []
    for i in range(0, len(dataset)):
        review = re.sub('[^a-zA-Z]', ' ', dataset[review_field][i])
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
        review = ' '.join(review)
        corpus.append(review)

    cv = CountVectorizer(max_features = 400)
    X = cv.fit_transform(corpus).toarray()
    
    # Si es para entrenar modelo habrá Y. Si queremo clasificar no
    if train:
        Y = dataset.iloc[:, 1].values
        return X, Y
    else:
        return X


df_comentarios

NameError: name 'df_comentarios' is not defined

In [15]:
# Aquí obtenemos nuestros datos X e Y
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist, classify, NaiveBayesClassifier
from nltk.tokenize import word_tokenize
X, Y = get_data_x_y(dataset, 'Review', train=True)
X_comentarios_test = get_data_x_y(df_comentarios, 'revs')

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/smartin/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [17]:
# Ahora dividimos entre datos de entrenamiento y test primero.
# Luego los propios datos de entreno los dividimos en entreno, propiamente dicho
# y validación
from sklearn.model_selection import train_test_split
x_2, x_test, y_2, y_test = train_test_split(X, Y, test_size = 0.15, random_state = 42)
x_train, x_val, y_train, y_val = train_test_split(x_2, y_2, test_size = 0.15, random_state = 42)

NameError: name 'X' is not defined

In [18]:
# Vemos cómo han quedado repartidos nuestros datos
print('Tamaño muestra entreno',len(x_train))
print('Tamaño muestra validación',len(x_val))
print('Tamaño muestra test',len(x_test))

NameError: name 'x_train' is not defined

Con shape vemos la forma de nuestros datos. En este caso tenemos 722 arrays y cada uno cuenta con un array de 400 datos.

In [19]:
x_train.shape

NameError: name 'x_train' is not defined

In [20]:
len(y_train)

NameError: name 'y_train' is not defined

In [21]:
# Modelo en Keras
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(len(x_train), 6))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(6, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model.compile(optimizer="adam",loss='binary_crossentropy', metrics=['accuracy'])
history=model.fit(x_train, y_train ,epochs=100, batch_size=512, validation_data=(x_val,y_val))

NameError: name 'x_train' is not defined

In [ ]:
Y_comentarios_pred = classifier.predict(X_comentarios_test)
len(Y_comentarios_pred)

eq = {0:'Negativo',1:'Positivo'}
print(df_comentarios['revs'].iloc[100])
print('Su clasificación es', eq[Y_comentarios_pred[100]])

NameError: ignored

Vemos como de bueno ha sido el entreno respecto a los datos de test. Que en este caso no muy allá...

In [ ]:
results = model.evaluate(x_test,y_test)
print('Resultados: ')
print(dict(zip(model.metrics_names, results)))



5/5 [==============================] - 0s 3ms/step - loss: 0.6928 - accuracy: 0.5267
Resultados: 
{'loss': 0.6927876472473145, 'accuracy': 0.5266666412353516}


Probamos a clasificar una de las muestras de test y ver cuál era el sentimiento esperado.

In [ ]:
pred = model.predict(x_test, batch_size=512, verbose=1)  
predicted = np.argmax(pred, axis=1)

print('Clasificación del primer comentario de los test: ' + str(np.argmax(y_test[0])))  
print('Predicción:         ' + str(predicted[0]))  

1/1 [==============================] - 0s 88ms/step
Clasificación del primer comentario de los test: 0
Predicción:         0


Ahora probamos a clasificar una de las muestras que extragimos antes.

Este tipo de problemas se pueden solucionar también con Machine Learning convencional. Por ejemplo vamos a ver una Multinomial con Naive Bayes, una Bernoulli Naive Bayes y una Regresión Logística, tipos de algoritmos de Machine Learning.

## Metodos de Machine Learning

In [ ]:
# Multinomial NB

# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import MultinomialNB
classifier_m = MultinomialNB(alpha=0.1)
classifier_m.fit(x_train, y_train)

# Predicting the Test set results
y_pred = classifier_m.predict(x_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred)
score3= recall_score(y_test,y_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[52 19]
 [25 54]]


Accuracy is  70.67 %
Precision is  0.74
Recall is  0.68


In [ ]:
# Bernoulli NB

# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import BernoulliNB
classifier_b = BernoulliNB(alpha=0.8)
classifier_b.fit(x_train, y_train)

# Predicting the Test set results
y_pred = classifier_b.predict(x_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred)
score3= recall_score(y_test,y_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[54 17]
 [28 51]]


Accuracy is  70.0 %
Precision is  0.75
Recall is  0.65


In [ ]:
# Logistic Regression

# Fitting Logistic Regression to the Training set
from sklearn import linear_model
classifier_lr = linear_model.LogisticRegression(C=1.5)
classifier_lr.fit(x_train, y_train)

# Predicting the Test set results
y_pred = classifier_lr.predict(x_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred)
score3= recall_score(y_test,y_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[58 13]
 [33 46]]


Accuracy is  69.33 %
Precision is  0.78
Recall is  0.58


Ahora vamos a clasificar nuestros comentarios

In [ ]:
y_pred_lr = classifier_lr.predict(X_comentarios_test)
y_pred_b = classifier_b.predict(X_comentarios_test)
y_pred_m = classifier_m.predict(X_comentarios_test)

fila = 0
comentario = df_comentarios['revs'].iloc[fila]
print('El comentario:',comentario)
print('Con una regresión logísta su clasificación es:', y_pred_lr[fila])
print('Con una de Bernoulli su clasificación es:', y_pred_b[fila])
print('Con una Multinomial su clasificación es:', y_pred_lr[fila])

El comentario: Had a bit of trouble finding a good fine dining restaurant open in Tenerife with COVID and without having to travel to the north. Ended up finding this gem and ate here several times during our stay. Restaurants food is well above the standard of surrounding eateries and while not to the standard of great restaurants in capital cities it is a great choice if you want a fine dining experience while on holidays. The wine list is a little sparse but had some good quality Spanish wines on it. The eel dish was superb and the truffled egg was well done. 
Con una regresión logísta su clasificación es: 1
Con una de Bernoulli su clasificación es: 0
Con una Multinomial su clasificación es: 1


## Librerías especializadas en lenguaje natural

A su vez hay librerías que trabajan en concreto con sentimientos de palabras por nosotros. Por ejemplo, nltk tiene una amplia gama de bibliotecas para tratar lenguaje natural. No solamente para ayudarnos a clasificar sentimientos sino además proporcionan otro tipo de ayudas para tratar el lenguaje natural y diferentes idiomas además de inglés.

![PLN.jpg](https://drive.google.com/uc?export=view&id=16_1pwZlRVjrj1ooHsHYh8wpjv1GCV6LT)

Aquí un ejemplo de código para probar lo mismo de antes.

In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('twitter_samples')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist, classify, NaiveBayesClassifier
from nltk.tokenize import word_tokenize

corpus = []



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
import re, string, random

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

if __name__ == "__main__":

    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

    print("Accuracy is:", classify.accuracy(classifier, test_data))

    print(classifier.show_most_informative_features(10))

    custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."

    custom_tokens = remove_noise(word_tokenize(custom_tweet))

    print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]
Accuracy is: 0.9973333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2059.0 : 1.0
                      :) = True           Positi : Negati =   1612.7 : 1.0
                follower = True           Positi : Negati =     40.4 : 1.0
                     sad = True           Negati : Positi =     25.3 : 1.0
                followed = True           Negati : Positi =     20.6 : 1.0
                     via = True           Positi : Negati =     19.0 : 1.0
                   enjoy = True           Positi : Negati =     16.5 : 1.0
                  arrive = True           Positi : Negati =     15.6 : 1.0
              appreciate = True           Positi : Negati =     15.5 : 1.0
                   didnt = True           Negati : Positi =     14.5 : 1.0
None
I ordered just once from TerribleCo, 

Prueba con nuestros datos guardados del scrapeo


In [ ]:
review = df_comentarios['revs'].iloc[0]
custom_tokens = remove_noise(word_tokenize(review))

print("Comentario:",review)
print("Clasificación que nos devuelve:",classifier.classify(dict([token, True] for token in custom_tokens)))

NameError: ignored